- Nick Clifford
- April 30, 2020
- SYS 6016

# ML Final: Data Pipeline

# Setup

In [1]:
import pandas as pd
import numpy as np
pd.options.display.max_columns = 20

datadir = '/Users/nickclifford/Documents/UVA/Spring 2020/SYS 6016 Machine Learning/final_proj/data'

# Team Data 

In [2]:
# read in team data
bask = pd.read_csv(datadir + '/updated_bask_data2.csv', parse_dates=['date'])

In [3]:
# remove playoff season games, reset index
bask = bask[pd.isnull(bask.playoff)]
bask.index = range(len(bask))

# drop playoff, elo_pre, and score columns
bask = bask.drop(['playoff', 'elo1_pre', 'elo2_pre', 'score1', 'score2'], axis=1)

# convert home/away team cols to a binary col for home for the team of interest
bask['home'] = pd.Series(np.where(bask['team1..Home.'] == bask.team_of_interest, 1, 0))

# rename team_of_interest to team
bask = bask.rename({'team_of_interest':'team'}, axis=1)

# All-star Data 

In [4]:
# read in all-star data
stars = pd.read_excel(datadir + '/All Star List.xlsx')

In [5]:
# turn every 4 columns into df, remove NaN values from each, rename columns to same heading, add a date column for year of all-star game
df16 = stars.iloc[:-2,:4].rename({'2015 2016 All Stars': 'player', 'Team':'team', 'Starter?':'starter', 'Age':'age'}, axis=1)
df16['season'] = 2016

df17 = stars.iloc[:-2,4:8].rename({'2016 2017 All Stars': 'player', 'Team.1':'team', 'Starter?.1':'starter', 'Age.1':'age'}, axis=1)
df17['season'] = 2017

df18 = stars.iloc[:-3,8:12].rename({'2017 2018 All Stars': 'player', 'Team.2':'team', 'Starter?.2':'starter', 'Age.2':'age'}, axis=1)
df18['season'] = 2018

df19 = stars.iloc[:,12:16].rename({'2018 2019 All Stars': 'player', 'Team.3':'team', 'Starter? ':'starter', 'Age.3':'age'}, axis=1)
df19['season'] = 2019

df20 = stars.iloc[:-2,16:20].rename({'2019 2020 All Stars': 'player', 'Team.4':'team', 'Starter?.3':'starter', 'Age.4':'age'}, axis=1)
df20['season'] = 2020

In [6]:
# reformat table into long format
stars = pd.concat([df16, df17, df18, df19, df20])
#stars['season'] = pd.to_datetime(stars.season, format='%Y')

# fix misspelled player names and lowercase everything
stars['player'] = stars.player.str.strip()
stars.player.replace('Bradly Beal', 'Bradley Beal', inplace=True)
stars.player.replace('Ressell Westbrook', 'Russell Westbrook', inplace=True)
stars.player.replace('Ressell Westbrook', 'Russell Westbrook', inplace=True)
stars['player'] = stars.player.str.lower()

# add birthday column as opposed to using age as feature 
stars['born'] = stars.season - stars.age

# Injury Data

In [7]:
# read in injury data
miss = pd.read_csv(datadir + '/abscences.csv', index_col=0, parse_dates=['date'])

In [8]:
# lowercase all player names
miss['player'] = miss.player.str.lower()


team_names = {'76ers':'PHI', 'Blazers':'POR', 'Bucks':'MIL', 'Bulls':'CHI', 
              'Cavaliers':'CLE', 'Celtics':'BOS', 'Clippers':'LAC',  'Grizzlies':'MEM', 
              'Hawks':'ATL', 'Heat':'MIA', 'Hornets':'CHO','Jazz':'UTA', 'Kings':'SAC', 
              'Knicks':'NYK', 'Lakers':'LAL', 'Magic':'ORL', 'Mavericks':'DAL', 'Nets':'BRK',
              'Nuggets':'DEN', 'Pacers':'IND', 'Pelicans':'NOP', 'Pistons':'DET', 
              'Raptors':'TOR', 'Rockets':'HOU', 'Spurs':'SAS', 'Suns':'PHO', 'Thunder':'OKC', 
              'Timberwolves':'MIN', 'Warriors':'GSW', 'Wizards':'WAS'}

# remove rows where team is NaN or 'Bullets'
miss = miss[pd.notnull(miss.team)]
miss = miss[miss.team != 'Bullets'] 

# replace team name with abbreviation to match other tables
miss['team'] = miss.team.replace(team_names)

# add a season column according date of the action
# regular_season date dictionary: {'season year': ['startdate','endate']}
reg_dates = {'2016': [pd.to_datetime('2015-10-27'), pd.to_datetime('2016-04-13')],
             '2017':[pd.to_datetime('2016-10-25'),pd.to_datetime('2017-04-12')],
             '2018':[pd.to_datetime('2017-10-17'),pd.to_datetime('2018-04-11')],
             '2019':[pd.to_datetime('2018-10-16'),pd.to_datetime('2019-04-13')],
             '2020':[pd.to_datetime('2019-10-22')]}

# new season starts on beginning of the regular season
miss['season'] = np.where(miss.date < reg_dates['2020'][0], 2019, 2020)
miss['season'] = np.where(miss.date < reg_dates['2019'][0], 2018, miss['season'])
miss['season'] = np.where(miss.date < reg_dates['2018'][0], 2017, miss['season'])
miss['season'] = np.where(miss.date < reg_dates['2017'][0], 2016, miss['season'])

# Combine team & player

First we input an all-start name and recover their nba schedule from seasons 2015 to 2020. Then we find which days the player was moved to and from the active roster. We create features for games which we know the players are absent (injury, suspension, etc.) and labels for the games that players DNP for rest (located in injury notes). The result is a individual player datatable, with team/player features as columns and game observation rows. 

In [40]:
def get_data(player_name):
    """Given a player name, output a combined table of player and table data 
    including fields for injury/suspension and rest labels"""
    
    player_name = player_name.lower()
    
    # get the birthday of the player 
    born = stars.groupby('player').get_group(player_name).born.value_counts().index[0]
    
    # subset injury data to that of the player
    if player_name == 'john wall':
        df_miss = miss.query('player == "john wall (hildred)"')
    elif player_name == 'domantas sabonis':
        df_miss = miss.query('player == "domantas sabonis / domas sabonis"')
    elif player_name == 'bam adebayo':
        df_miss = miss.query('player == "edrice adebayo / bam adebayo"')
                
    else:
        df_miss = miss.query('player == @player_name')

    # get combination of team and the season year for the player
    player_seasons = df_miss.groupby(['season', 'team']).size().index.to_list()
    
    # get list of dates games that player played according to the team they played with that season    
    df_bask = bask.set_index(['season','team']).loc[player_seasons].reset_index()
    game_dates = df_bask.date.to_list()
    
    # acquired dates
    doA = df_miss.query('action == "acquired"')['date']
    # relinquished dates
    doR = df_miss.query('action == "relinquished"')['date']

    # find the dates that the player rested and did not play
    rest_dates = df_miss[(df_miss.notes.str.contains('rest ')) & (df_miss.action == 'relinquished')]['date'].to_list()
    rest_array = np.isin(game_dates, rest_dates).astype(int)
    
    reg_dates = {'2016': [pd.to_datetime('2015-10-27'), pd.to_datetime('2016-04-13')],
             '2017':[pd.to_datetime('2016-10-25'),pd.to_datetime('2017-04-12')],
             '2018':[pd.to_datetime('2017-10-17'),pd.to_datetime('2018-04-11')],
             '2019':[pd.to_datetime('2018-10-16'),pd.to_datetime('2019-04-13')],
             '2020':[pd.to_datetime('2019-10-22')]}

    df_temp = pd.DataFrame(pd.date_range(reg_dates['2016'][0], reg_dates['2020'][0], freq='D')).rename({0:'time'}, axis=1)
    df_temp['doG'] = np.isin(df_temp.time, game_dates)
    df_temp['doA'] = np.isin(df_temp.time, doA).astype(int)
    df_temp['doR'] = np.isin(df_temp.time, doR).astype(int)
    season_dates = pd.Series([item for sublist in list(reg_dates.values()) for item in sublist])
    df_temp['season'] = np.isin(df_temp.time, season_dates)
    df_temp['rest'] = np.isin(df_temp.time, pd.Series(rest_dates)).astype(int) 
    df_temp['absent'] = pd.Series(df_temp.doA + df_temp.season + df_temp['rest'] - df_temp.doR).replace({0, np.NaN})
    df_temp['absent'] = df_temp['absent'].ffill().replace({1:0, -1:1})
        
    df_bask['player'] = player_name
    df_bask['born'] = born
    df_bask['absent'] = df_temp['absent']
    df_bask['rest'] = rest_array

    return df_bask


## Concatenate data from all players

### Filter & Explicitly Defined

Subset data to seasons before 2018-2019 and ignore all-stars who were traded in the middle the those seasons. Also define rest games from notes that explicitly use the word 'rest '.

In [43]:
omit_players = ['jimmy butler', 'andre drummond', 'carmelo anthony', 'dwanye wade', 'blake griffin', 
                'marc gasol', 'pau gasol', 'isaiah thomas', 'demarcus cousins', 'deandre jordan', 'dangelo russell']

appended_data = []
for player in pd.unique(stars.player):
    if player not in omit_players:
        df = get_data(player)
        appended_data.append(df)
    

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:49: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


In [47]:
df_all = pd.concat(appended_data, axis=0)
df_all.to_csv(datadir + '/df_all.csv')

## Summary statistics

The data is heavily biased, with only 50 observations of rest

In [65]:
df_all.rest.value_counts()

0    5608
1      50
Name: rest, dtype: int64

In [69]:
df_all.describe()

,season,elo_prob1,elo_prob2,b2b,days_rest,df2.playoff_prob,miles,home,born,absent,rest
count,5658.000000,5658.000000,5658.000000,5658.000000,5658.000000,5658.000000,5658.000000,5658.000000,5658.000000,5658.000000,5658.000000
mean,2017.956522,0.612881,0.387119,0.357900,-0.398551,0.840232,583.116826,0.500000,1988.565217,0.011842,0.008837
std,1.398205,0.215160,0.215160,0.479425,82.693975,0.291460,586.875191,0.500044,3.281675,0.108183,0.093598
min,2016.000000,0.064858,0.018315,0.000000,-1632.000000,0.000000,0.000000,0.000000,1979.000000,0.000000,0.000000
25%,2017.000000,0.447325,0.204151,0.000000,2.000000,0.860000,0.000000,0.000000,1986.000000,0.000000,0.000000
50%,2018.000000,0.642313,0.357687,0.000000,2.000000,0.990000,430.000000,0.500000,1989.000000,0.000000,0.000000
75%,2019.000000,0.795849,0.552675,1.000000,2.000000,0.990000,937.000000,1.000000,1991.000000,0.000000,0.000000
max,2020.000000,0.981685,0.935142,1.000000,197.000000,1.000000,2708.000000,1.000000,1994.000000,1.000000,1.000000


# Data Combining V2

## Read in

In [57]:
from functools import reduce
from glob import glob
aneeshdir = '/Users/nickclifford/Documents/UVA/Spring 2020/SYS 6016 Machine Learning/final_proj/data/aneesh_data/'

In [122]:
file_list = glob(aneeshdir + '*')
file_list

['/Users/nickclifford/Documents/UVA/Spring 2020/SYS 6016 Machine Learning/final_proj/data/aneesh_data/age_layer.csv',
 '/Users/nickclifford/Documents/UVA/Spring 2020/SYS 6016 Machine Learning/final_proj/data/aneesh_data/date_of_game_layer.csv',
 '/Users/nickclifford/Documents/UVA/Spring 2020/SYS 6016 Machine Learning/final_proj/data/aneesh_data/participated_layer.csv',
 '/Users/nickclifford/Documents/UVA/Spring 2020/SYS 6016 Machine Learning/final_proj/data/aneesh_data/team_playoff_probabilities_layer.csv',
 '/Users/nickclifford/Documents/UVA/Spring 2020/SYS 6016 Machine Learning/final_proj/data/aneesh_data/opponent_playoff_probabilities_layer.csv',
 '/Users/nickclifford/Documents/UVA/Spring 2020/SYS 6016 Machine Learning/final_proj/data/aneesh_data/opponent_layer.csv',
 '/Users/nickclifford/Documents/UVA/Spring 2020/SYS 6016 Machine Learning/final_proj/data/aneesh_data/time_elapsed_layer.csv',
 '/Users/nickclifford/Documents/UVA/Spring 2020/SYS 6016 Machine Learning/final_proj/data/an

In [167]:
# all the player names that are the column headers in each file
col_names = ['John Wall', 'Khris Middleton', 'Nikola Vucevic',
       'Devin Booker', 'Domantas Sabonis', 'Andre Drummond',
       'LaMarcus Aldridge', 'Ben Simmons', 'Jimmy Butler', 'LeBron James',
       'Dirk Nowitzki', 'Luka Doncic', 'Blake Griffin', 'Damian Lillard',
       'Chris Paul', 'Jayson Tatum', 'Marc Gasol', 'Al Horford',
       'Giannis Antetokounmpo', 'Bam Adebayo', 'Dwyane Wade', 'Brandon Ingram',
       'Bradley Beal', 'Pau Gasol', 'Gordon Hayward', 'Karl-Anthony Towns',
       'Paul George', 'Kevin Durant', 'Rudy Gobert', 'Goran Dragic',
       'Kawhi Leonard', 'Kyrie Irving', 'Russell Westbrook', 'James Harden',
       'Paul Millsap', 'Victor Oladipo', 'Isaiah Thomas', 'Draymond Green',
       'Carmelo Anthony', 'Kemba Walker', 'Joel Embiid', 'Nikola Jokic',
       'Donovan Mitchell', 'DeAndre Jordan', 'D\'Angelo Russell',
       'Pascal Siakam', 'DeMarcus Cousins', 'Kobe Bryant', 'Stephen Curry',
       'Kyle Lowry', 'DeMar DeRozan', 'Trae Young', 'Anthony Davis',
       'Klay Thompson']

# Read in each file and melt from wide to long format
long_dfs = []
for file in file_list:
    df_wide = pd.read_csv(file).rename({'Unnamed: 0':'game'}, axis=1)
    value_name = file.split('/')[-1].rstrip('_layer.csv')
    df_long = pd.melt(df_wide, id_vars='game', var_name='player', value_vars=col_names, value_name=value_name).set_index(['player', 'game'])
    long_dfs.append(df_long)
    

df_all2 = reduce(lambda  left,right: pd.merge(left,right,on=['player', 'game'], how='outer'), long_dfs).reset_index()

# rename columns
df_all2 = df_all2.rename({'game':'game-season', 'ag':'age', 'date_of_gam':'date', 'participated':'rest', 'team_playoff_probabiliti':'df2.playoff_prob', 
                'opponent_playoff_probabiliti':'oppo_playoff_prob', 'time_elapsed':'days_rest', 
                          'distance_traveled':'miles', 'home_gam':'home'}, axis=1).reset_index()
# add season and game fields
df_all2['game'] = df_all2['game-season'].str.split().str[1]
df_all2['season'] = df_all2['game-season'].str.split().str[-2]
df_all2['date'] = pd.to_datetime(df_all2.date) # pd.datetime object
df_all2['rest'] = df_all2.rest.replace({1:-1}).replace({0:1}).replace({-1:0}) # change rest day == 1, nonrest == 0
df_all2['b2b'] = np.where(df_all2.date.diff() == pd.Timedelta('1 days 00:00:00'), 1, 0) # back to back col

# rearrange columns
df_all2 = df_all2[['player', 'date', 'season', 'game', 'opponent', 'age', 
                   'df2.playoff_prob', 'oppo_playoff_prob', 'days_rest', 'miles', 'home', 'b2b','rest']]

df_all2 = df_all2[pd.notnull(df_all2['rest'])]
#df_all2.to_csv(datadir + '/df_all2.csv')

In [164]:
df_all2

,player,date,season,game,opponent,age,df2.playoff_prob,oppo_playoff_prob,days_rest,miles,home,brb,rest
0,John Wall,2015-10-28,2016,1,ORL,25.141684,66.0,43.0,10.0,759.0,0.0,0,0.0
1,John Wall,2015-10-30,2016,2,MIL,25.147159,66.0,34.0,2.0,1067.0,0.0,0,0.0
2,John Wall,2015-10-31,2016,3,NYK,25.149897,66.0,6.0,1.0,635.0,1.0,1,0.0
3,John Wall,2015-11-04,2016,4,SAS,25.160849,68.0,98.0,4.0,0.0,1.0,0,0.0
4,John Wall,2015-11-06,2016,5,BOS,25.166324,68.0,84.0,2.0,394.0,0.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21109,Klay Thompson,2019-11-11,2020,59,UTA,29.754962,14.0,94.0,2.0,1379.0,1.0,0,1.0
21110,Klay Thompson,2019-11-13,2020,60,LAL,29.760438,14.0,83.0,2.0,344.0,0.0,0,1.0
21111,Klay Thompson,2019-11-15,2020,61,BOS,29.765914,3.0,99.9,2.0,344.0,1.0,0,1.0
21112,Klay Thompson,2019-11-17,2020,62,NOP,29.771389,3.0,14.0,2.0,1917.0,0.0,0,1.0


## Summary statistics

This data more loosely defines rest. Specifically, if players are inactive for short periods of time, or did not suffer from season-long inactivity, we label the games that they did not play as 'rest'

In [118]:
df_all2.rest.value_counts()

1.0    15663
0.0     3635
Name: rest, dtype: int64

In [119]:
df_all2.describe()

,age,df2.playoff_prob,oppo_playoff_prob,days_rest,miles,home,rest
count,19298.000000,19298.000000,19298.000000,19298.000000,19298.000000,19298.000000,19298.000000
mean,27.505228,67.569784,52.531812,2.213183,561.304177,0.499534,0.811639
std,4.466052,37.976378,40.719080,1.297107,548.237036,0.500013,0.391011
min,18.992471,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
25%,24.043806,34.000000,5.000000,2.000000,0.000000,0.000000,1.000000
50%,27.370294,88.000000,58.000000,2.000000,442.000000,0.000000,1.000000
75%,30.406571,99.900000,98.000000,2.000000,879.000000,1.000000,1.000000
max,40.807666,100.000000,100.000000,10.000000,2708.000000,1.000000,1.000000
